In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [5]:
def view_detections():    
    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [6]:
view_detections()

In [7]:
#extracting the x,y co-ordinates of landmarks detected by mediapipe and storing in numpy arrays, here left and right hands with the posture
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,left_hand,right_hand])

In [14]:
# Actions that we try to detect | null implies none of the meaningful actions were detected in current frames
actions = np.array(['null','apply-throttle', 'release-throttle'])

# Path for exported data, numpy arrays
DATA_PATH = os.path.join(os.path.dirname(os.getcwd()),"MP_Data") 

#next batch additional size
seq_batch = 5

#the size of last sequence of videos batch
dirmax = np.max(np.array([eval(i) for i in os.listdir(os.path.join(DATA_PATH, actions[0]))]))

# total current sequences plus extra batch size
sequence_limit = dirmax + seq_batch + 1

# Videos are going to be 30 frames in length
frame_limit = 30


In [30]:
#This block creates the respective folders to store the data(the numpy files)
# each action(ex: throttle-apply) -> video(sequence) -> frames(each one is a .np file containing x,y coordinates of the landmarks)
for action in actions:
    for sequence in range(dirmax+1,sequence_limit):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [10]:
# it may be required to set a diffferent limit for sequences than the limit set above when making the folders
sequence_limit = 20 

In [ ]:
cap = cv2.VideoCapture(0)

is_recording = False  # Flag to track recording state
is_done = False  # Flag to track the initial pause

# Keyboard event handler
def key_press(key):
    global is_recording,is_done
    #click s to set specific flags
    if key == ord(' '):
        is_done = False
        is_recording = True

#copying the actions array to a temp array to slice to train for a few actions if required 
actions_temp = actions.copy()
actions_temp = actions_temp[2:3]
#Start value for sequence
start_seq = 15
#Intial values of frame and video number
sequence = start_seq
frame_num = 0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # Display "Collecting..." message after pressing 's'
        if frame_num >=1 and is_recording:
            cv2.putText(image, "Collecting - frame {}".format(frame_num), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4,cv2.LINE_AA)
        if is_done:
            cv2.putText(image, 'action: {} | sequence: {} '.format(actions_temp[-1] if len(actions_temp) else 'empty',sequence), (20,20), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.632, (0,255, 0), 4, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)
        #taking input from user
        key_selected = cv2.waitKey(1)
        # Store landmarks when 's' key is pressed
        key_press(key_selected)
        #click "s" to initiate video/frame collection
        if  is_recording and len(actions)>0:
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, actions_temp[-1], str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)
                    print('action: {} | sequence: {} | frame: {} '.format(actions_temp[-1] if len(actions_temp) else 'empty',sequence,frame_num))
                    frame_num += 1
                    if frame_num == frame_limit:
                        frame_num=0
                        is_recording = False
                        is_done = True
                        sequence+=1
                    if sequence==sequence_limit:
                        sequence=start_seq
                        actions_temp = actions_temp[:-1]
        elif key_selected == ord('q'):
            break
        else:
            pass
        
    cap.release()
    cv2.destroyAllWindows()

In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [15]:
label_map = {label:num for num, label in enumerate(actions)}

In [16]:
label_map

{'null': 0, 'apply-throttle': 1, 'release-throttle': 2}

In [27]:
sequences, labels = [], []
for action in actions_temp:
    for sequence in range(sequence_limit):
        window = []
        for frame_num in range(frame_limit):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
X = np.array(sequences)

In [29]:
y = to_categorical(labels).astype(int)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [31]:
X.shape

(60, 30, 258)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [33]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions_temp.shape[0], activation='relu'))

In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [38]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 237,251
Trainable params: 237,251
Non-trai

In [39]:
model.save(os.path.join(os.path.dirname(os.getcwd()),"Models","actions.h5"))

In [33]:
model.load_weights(os.path.join(os.path.dirname(os.getcwd()),"Models","actions.h5") )

In [21]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [22]:
yhat = model.predict(X_test)

NameError: name 'X_test' is not defined

In [43]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [44]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 0],
        [0, 1]],

       [[2, 0],
        [0, 1]],

       [[2, 0],
        [0, 1]]], dtype=int64)

In [45]:
accuracy_score(ytrue, yhat)

1.0

In [34]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.75

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions_temp[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions_temp[np.argmax(res)])
                else:
                    sentence.append(actions_temp[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions_temp, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()